In [1]:
!pip install adversarial-robustness-toolbox torch matplotlib numpy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 23.2 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.4 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 107.6 MB/s eta 0:00:0000:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 80.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 53.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 8.8 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 34.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 15.0 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 9.2 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 100.5 MB/s eta 0:00:0000:0100:01
  Attempting uninstall:

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, TensorDataset, ConcatDataset
import torchvision.models as models
from torchvision.models import resnet18
import numpy as np
import os
from tqdm import tqdm

class AddGaussianNoise(object):
    def __init__(self, mean=0., std=0.01):
        self.std = std
        self.mean = mean
        
    def __call__(self, tensor):
        return tensor + torch.randn(tensor.size()) * self.std + self.mean
    
    def __repr__(self):
        return self.__class__.__name__ + f'(mean={self.mean}, std={self.std})'

In [3]:
EPOCHS_DETECTION = 5
BATCH_SIZE = 64
LEARNING_RATE = 0.001

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Initialize Model
baseline_model_resnet = models.resnet18(weights='IMAGENET1K_V1')

num_ftrs = baseline_model_resnet.fc.in_features
baseline_model_resnet.fc = nn.Linear(num_ftrs, 2) 

if torch.cuda.device_count() > 1:
    baseline_model_resnet = torch.nn.DataParallel(baseline_model_resnet)

baseline_model_resnet = baseline_model_resnet.to(device)
print("Model initialized for Binary Classification (2 classes)")

Using device: cuda


Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 193MB/s]


Model initialized for Binary Classification (2 classes)


In [4]:
transform_clean = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

transform_adv = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

print("Transforms Ready: Separate processing for Clean vs. Adv data.")

Transforms Ready: Separate processing for Clean vs. Adv data.


In [ ]:
# loading folder of images and giving binary labels
class BinaryLabelDataset(torch.utils.data.Dataset):
    def __init__(self, root, label, transform):
        self.dataset = datasets.ImageFolder(root=root, transform=transform)
        self.label = label
    
    def __getitem__(self, index):
        data, _ = self.dataset[index]
        return data, self.label
    
    def __len__(self):
        return len(self.dataset)

CLEAN_PATH = '/kaggle/input/imagenet100/train.X1' 
ADV_PATH = '/kaggle/input/imagenet100-adversarial/train'

clean_ds = BinaryLabelDataset(
    root=CLEAN_PATH,
    label=0, 
    transform=transform_clean 
)

if os.path.exists(ADV_PATH):
    adv_ds = BinaryLabelDataset(
        root=ADV_PATH, 
        label=1, 
        transform=transform_adv
    )
    full_dataset = ConcatDataset([clean_ds, adv_ds])
    print(f"Loaded Clean ({len(clean_ds)}) + Adversarial ({len(adv_ds)})")
else:
    print(f"WARNING: Adversarial path {ADV_PATH} not found.")
    full_dataset = clean_ds

train_loader = DataLoader(full_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=2)
print(f"Total training images: {len(full_dataset)}")

Loaded Clean (32500) + Adversarial (32500)
Total training images: 65000


In [6]:
print("Folders in /kaggle/input:")
print(os.listdir("/kaggle/input"))

Folders in /kaggle/input:
['imagenet100', 'imagenet100-adversarial']


In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(baseline_model_resnet.parameters(), lr=LEARNING_RATE, momentum=0.9)

scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.1)

print("Model configured (Optimizer: SGD, Loss: CrossEntropy, Scheduler: StepLR).")

Model configured (Optimizer: SGD, Loss: CrossEntropy, Scheduler: StepLR).


In [ ]:
def train_model(model, train_loader, optimizer, loss_fn, epochs, description="", scheduler=None):
    model.train()
    
    for epoch in range(epochs):
        running_loss = 0.0
        correct = 0
        total = 0

        loop = tqdm(train_loader, desc=f"[{description}] Epoch {epoch+1}/{epochs}", leave=True)

        for images, labels in loop:
            images, labels = images.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(images)
            
            loss = loss_fn(outputs, labels)
            
            loss.backward()
            optimizer.step()

            running_loss += loss.item() * images.size(0)
            
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()

            loop.set_postfix(loss=loss.item(), acc=100.0 * correct / total)

        if scheduler:
            scheduler.step()

        epoch_loss = running_loss / len(train_loader.dataset)
        epoch_acc = 100.0 * correct / total

        print(f" -> Epoch {epoch+1} Finished | Avg Loss: {epoch_loss:.4f} | Avg Acc: {epoch_acc:.2f}%")

    print("Training complete.")

In [ ]:
print("--- Training ImageNet-100 ResNet Detection Model ---")

train_model(
    model=baseline_model_resnet,
    train_loader=train_loader, 
    optimizer=optimizer,
    loss_fn=criterion,
    epochs=EPOCHS_DETECTION, 
    description="ImageNet-100 ResNet Detector",
    scheduler=scheduler 
)

print("--- Detection Model Training Finished ---")

model_save_dir = 'models'
os.makedirs(model_save_dir, exist_ok=True) 
model_save_path = "detection_resnet_imagenet100.pth" 

if isinstance(baseline_model_resnet, torch.nn.DataParallel):
    torch.save(baseline_model_resnet.module.state_dict(), model_save_path)
else:
    torch.save(baseline_model_resnet.state_dict(), model_save_path)

print(f"ImageNet-100 ResNet Detection model saved to: {model_save_path}")

--- Training ImageNet-100 ResNet Detection Model ---


[ImageNet-100 ResNet Detector] Epoch 1/5: 100%|██████████| 1016/1016 [05:31<00:00,  3.07it/s, acc=99.3, loss=0.00748] 


 -> Epoch 1 Finished | Avg Loss: 0.0227 | Avg Acc: 99.30%


[ImageNet-100 ResNet Detector] Epoch 2/5: 100%|██████████| 1016/1016 [03:25<00:00,  4.95it/s, acc=100, loss=0.000237]


 -> Epoch 2 Finished | Avg Loss: 0.0022 | Avg Acc: 99.97%


...

 -> Epoch 3 Finished | Avg Loss: 0.0011 | Avg Acc: 99.99%


[ImageNet-100 ResNet Detector] Epoch 4/5: 100%|██████████| 1016/1016 [03:25<00:00,  4.95it/s, acc=100, loss=0.000311]


 -> Epoch 4 Finished | Avg Loss: 0.0008 | Avg Acc: 99.99%


[ImageNet-100 ResNet Detector] Epoch 5/5: 100%|██████████| 1016/1016 [03:24<00:00,  4.97it/s, acc=100, loss=4.42e-5] 

 -> Epoch 5 Finished | Avg Loss: 0.0006 | Avg Acc: 99.99%
Training complete.
--- Detection Model Training Finished ---
ImageNet-100 ResNet Detection model saved to: detection_resnet_imagenet100.pth
